use loc[i, field] instead of iloc[i, field], iloc is very slow.

In [1]:
import sys
sys.path = ['/root/autodl-tmp/xiaolong/WorkSpace/Amazon-KDDCUP-23/RecStudio/'] + sys.path
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
import random
import numpy as np
import pandas as pd
import cudf, itertools
import scipy.sparse as ssp
from functools import lru_cache, partial
from tqdm import tqdm, trange
from collections import Counter, defaultdict
import torch
import pickle

/root/autodl-tmp/xiaolong/.conda/envs/torch12/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def cast_dtype(df : pd.DataFrame, columns=None):
    if columns is None:
        columns = df.columns
    for k in columns:
        dt = type(df[k].iloc[0])
        if 'float' in str(dt):
            df[k] = df[k].astype('float32')
        elif 'int' in str(dt):
            df[k] = df[k].astype('int32')
        elif dt == list:
            dt_ = type(df.iloc[0][k][0])
            if 'float' in str(dt_):
                df[k] = df[k].apply(lambda x : np.array(x, dtype=np.float32))
            elif 'int' in str(dt_):
                df[k] = df[k].apply(lambda x : np.array(x, dtype=np.int32))

In [3]:
def _load_cache(path):
    with open(path, 'rb') as f:
        download_obj = pickle.load(f)
    return download_obj

In [4]:
def get_scores(merged_candidates_df, query_embeddings, product_embeddings):
    batch_size = 2048
    num_iter = (len(merged_candidates_df) - 1) // batch_size + 1
    score_list = []
    with torch.no_grad():
        for i in tqdm(range(num_iter)):
            st, ed = i * batch_size, (i + 1) * batch_size 
            batch_sess = merged_candidates_df.iloc[st : ed]
            batch_sess_id = torch.tensor(batch_sess['sess_id'].tolist(), dtype=torch.long, device=query_embeddings.device)
            batch_product_id = torch.tensor(batch_sess['dataset_id'].tolist(), dtype=torch.long, device=product_embeddings.device)
            query_emb = query_embeddings[batch_sess_id]
            product_emb = product_embeddings[batch_product_id]
            batch_score = (query_emb * product_emb).sum(dim=-1) 
            score_list.append(batch_score.cpu())
        score_list = torch.cat(score_list, dim=0).cpu().tolist()
        return score_list 

In [5]:
def normalize_scores(score_df, score_name, normalized_score_name):
    # score_df_g = cudf.from_pandas(score_df)
    score_df['exp_score'] = np.exp(score_df[score_name].to_numpy())
    scores_sum = score_df[['sess_id', 'exp_score']].groupby('sess_id').sum()
    scores_sum.reset_index(inplace=True)
    scores_sum = scores_sum.sort_values(by=['sess_id'], ascending=True)
    scores_sum.reset_index(drop=True, inplace=True)
    scores_sum.rename(columns={'exp_score' : 'score_sum'}, inplace=True)

    merged_score_df = score_df.merge(scores_sum, how='left', left_on=['sess_id'], right_on=['sess_id'])
    merged_score_df = merged_score_df.sort_values(by=['sess_id', 'product'])
    merged_score_df.reset_index(drop=True, inplace=True)
    
    # merged_score_df = merged_score_df_g.to_pandas(merged_score_df_g)
    score_df[normalized_score_name] = merged_score_df['exp_score'] / merged_score_df['score_sum']
    score_df['exp_score'] = merged_score_df['exp_score']
    score_df['score_sum'] = merged_score_df['score_sum']

    # del scores_sum_g
    # del merged_score_df_g 

# Merge test score

In [6]:
merged_candidates_feature_test_path = '/root/autodl-tmp/xiaolong/WorkSpace/Amazon-KDDCUP-23/XGBoost/candidates/merged_candidates_test_feature.parquet'
test_sessions_path = '/root/autodl-tmp/xiaolong/WorkSpace/Amazon-KDDCUP-23/raw_data/sessions_test_task1.csv'

In [7]:
DE_product_embeddings_path = '/root/autodl-tmp/xiaolong/WorkSpace/Amazon-KDDCUP-23/candidates/query_embeddings/SeqMLP/kdd_cup_2023_DE/product_embeddings_2023-05-24-17-28-00.pt'
DE_test_embeddings_path = '/root/autodl-tmp/xiaolong/WorkSpace/Amazon-KDDCUP-23/candidates/query_embeddings/SeqMLP/kdd_cup_2023_DE/predict_embeddings_2023-05-24-17-28-41.pt'
JP_product_embeddings_path = '/root/autodl-tmp/xiaolong/WorkSpace/Amazon-KDDCUP-23/candidates/query_embeddings/SeqMLP/kdd_cup_2023_JP/product_embeddings_2023-05-24-17-28-57.pt'
JP_test_embeddings_path = '/root/autodl-tmp/xiaolong/WorkSpace/Amazon-KDDCUP-23/candidates/query_embeddings/SeqMLP/kdd_cup_2023_JP/predict_embeddings_2023-05-24-17-29-40.pt'
UK_product_embeddings_path = '/root/autodl-tmp/xiaolong/WorkSpace/Amazon-KDDCUP-23/candidates/query_embeddings/SeqMLP/kdd_cup_2023_UK/product_embeddings_2023-05-24-17-29-53.pt'
UK_test_embeddings_path = '/root/autodl-tmp/xiaolong/WorkSpace/Amazon-KDDCUP-23/candidates/query_embeddings/SeqMLP/kdd_cup_2023_UK/predict_embeddings_2023-05-24-17-30-34.pt'

In [8]:
@lru_cache(maxsize=1)
def read_merged_candidates_feature_test():
    return pd.read_parquet(merged_candidates_feature_test_path, engine='pyarrow')

@lru_cache(maxsize=1)
def read_test_sessions():
    return pd.read_csv(test_sessions_path)

In [9]:
merged_candidates = read_merged_candidates_feature_test()
test_sessions = read_test_sessions()
EMBED_DIM = 256
merged_candidates.sort_values(by=['sess_id', 'product'], inplace=True)
merged_candidates.reset_index(drop=True, inplace=True)

In [10]:
# sess embeddings 
test_DE_query_emb = torch.load(DE_test_embeddings_path, map_location='cpu')
test_JP_query_emb = torch.load(JP_test_embeddings_path, map_location='cpu')
test_UK_query_emb = torch.load(UK_test_embeddings_path, map_location='cpu')
test_query_embeddings = torch.cat(
    [test_DE_query_emb[test_sessions['locale'] == 'DE'], test_JP_query_emb[test_sessions['locale'] == 'JP'], test_UK_query_emb[test_sessions['locale'] == 'UK']],
    dim=0)

In [12]:
# product embeddings 
DE_product_emb = torch.load(DE_product_embeddings_path, map_location='cpu')
JP_product_emb = torch.load(JP_product_embeddings_path, map_location='cpu')
UK_product_emb = torch.load(UK_product_embeddings_path, map_location='cpu')
product_embeddings = torch.cat([DE_product_emb, JP_product_emb, UK_product_emb], dim=0)

In [13]:
DE_dataset_cache = '/root/autodl-tmp/xiaolong/WorkSpace/Amazon-KDDCUP-23/.recstudio/cache/c76eddf0a07106ffcce7ce8010856a3b'
JP_dataset_cache = '/root/autodl-tmp/xiaolong/WorkSpace/Amazon-KDDCUP-23/.recstudio/cache/81a71d0a18766af84b3beab69bf53e69'
UK_dataset_cache = '/root/autodl-tmp/xiaolong/WorkSpace/Amazon-KDDCUP-23/.recstudio/cache/250dbc09c30162452e00486051e47756'
DE_train_dataset, DE_valid_dataset = _load_cache(DE_dataset_cache)
JP_train_dataset, JP_valid_dataset = _load_cache(JP_dataset_cache)
UK_train_dataset, UK_valid_dataset = _load_cache(UK_dataset_cache)
locale_map = {
    'DE' : DE_train_dataset.field2token2idx['product_id'], 
    'JP' : JP_train_dataset.field2token2idx['product_id'], 
    'UK' : UK_train_dataset.field2token2idx['product_id']
    }

In [14]:
DE_product_list, DE_id_list = list(zip(*locale_map['DE'].items()))
JP_product_list, JP_id_list = list(zip(*locale_map['JP'].items()))
UK_product_list, UK_id_list = list(zip(*locale_map['UK'].items()))
product_list = list(DE_product_list) + list(JP_product_list) + list(UK_product_list)
id_list = list(DE_id_list) + list(JP_id_list) + list(UK_id_list)
locale_list = ['DE'] * len(DE_id_list) + ['JP'] * len(JP_id_list) + ['UK'] * len(UK_id_list)
product_id_df = pd.DataFrame({'locale' : locale_list, 'product' : product_list, 'dataset_id' : id_list})

In [15]:
merged_candidates_seqmlp = merged_candidates[['sess_id', 'sess_locale', 'product']].copy()

In [16]:
# merged_candidates_seqmlp_g = cudf.from_pandas(merged_candidates_seqmlp)
# product_id_df_g = cudf.from_pandas(product_id_df)

In [17]:
# merged_candidates_seqmlp_score_g = merged_candidates_seqmlp_g.merge(product_id_df_g, how='left', left_on=['sess_locale', 'product'], right_on=['locale', 'product'])
# merged_candidates_seqmlp_score_g['dataset_id'] = merged_candidates_seqmlp_score_g['dataset_id'].fillna(0)
# merged_candidates_seqmlp_score_g.drop(columns=['locale'], inplace=True)
# merged_candidates_seqmlp_score_g = merged_candidates_seqmlp_score_g.sort_values(by=['sess_id', 'product'])
# merged_candidates_seqmlp_score_g.reset_index(drop=True, inplace=True)
# merged_candidates_seqmlp_score = merged_candidates_seqmlp_score_g.to_pandas()

In [ ]:
merged_candidates_seqmlp_score = merged_candidates_seqmlp.merge(product_id_df, how='left', left_on=['sess_locale', 'product'], right_on=['locale', 'product'])
merged_candidates_seqmlp_score['dataset_id'] = merged_candidates_seqmlp_score['dataset_id'].fillna(0)
merged_candidates_seqmlp_score.drop(columns=['locale'], inplace=True)
merged_candidates_seqmlp_score = merged_candidates_seqmlp_score.sort_values(by=['sess_id', 'product'])
merged_candidates_seqmlp_score.reset_index(drop=True, inplace=True)
assert len(merged_candidates_seqmlp_score) == len(merged_candidates)

In [18]:
# del merged_candidates_seqmlp_score_g
# del product_id_df_g
# del merged_candidates_seqmlp_g

In [19]:
locale_offset = {'DE' : 0, 'JP' : len(DE_product_list), 'UK' : len(DE_product_list) + len(JP_product_list)}
for locale in ['DE', 'JP', 'UK']:
    merged_candidates_seqmlp_score['dataset_id'][merged_candidates_seqmlp_score['sess_locale'] == locale] = \
        merged_candidates_seqmlp_score['dataset_id'][merged_candidates_seqmlp_score['sess_locale'] == locale] + locale_offset[locale]

/tmp/ipykernel_450910/1863069582.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_candidates_seqmlp_score['dataset_id'][merged_candidates_seqmlp_score['sess_locale'] == locale] = \


In [20]:
merged_candidates_seqmlp_score['seqmlp_scores'] = get_scores(merged_candidates_seqmlp_score, test_query_embeddings, product_embeddings)

100%|██████████| 33901/33901 [01:08<00:00, 498.50it/s]


In [21]:
normalize_scores(merged_candidates_seqmlp_score, 'seqmlp_scores', 'normalized_seqmlp_scores')

In [22]:
merged_candidates['seqmlp_scores'] = merged_candidates_seqmlp_score['seqmlp_scores']
merged_candidates['normalized_seqmlp_scores'] = merged_candidates_seqmlp_score['normalized_seqmlp_scores']

In [56]:
cast_dtype(merged_candidates, ['seqmlp_scores', 'normalized_seqmlp_scores'])
merged_candidates.to_parquet(merged_candidates_feature_test_path, engine='pyarrow')

In [25]:
merged_candidates[(merged_candidates['sess_id'] == 230000)].sort_values(by='normalized_sasrec_scores_2', ascending=False)[['sess_locale', 'product', 'normalized_sasrec_scores_2', 'sasrec_scores_2', 'normalized_seqmlp_scores', 'seqmlp_scores']][:20]

,sess_locale,product,sasrec_normalized_scores_3,sasrec_scores_3,seqmlp_normalized_scores,seqmlp_scores
50247381,UK,B093LPWTD8,0.084951,10.352390,0.091373,10.937450
50247327,UK,B073JJ9HP2,0.072020,10.187253,0.066935,10.626210
50247391,UK,B094R47H8Q,0.069588,10.152899,0.039884,10.108466
50247286,UK,B009WU0LWG,0.060782,10.017603,0.033926,9.946680
50247376,UK,B093C9B1HK,0.051335,9.848681,0.074143,10.728494
50247387,UK,B093LVB4P7,0.043121,9.674311,0.051726,10.368446
50247272,UK,B005EJFL42,0.029520,9.295372,0.026107,9.684687
50247273,UK,B005EJFLEM,0.028960,9.276229,0.04013,10.114613
50247375,UK,B093C88F2R,0.027596,9.227978,0.041181,10.140461
50247377,UK,B093CBBKG8,0.026901,9.202470,0.046459,10.261065
